# **Usando Python e dados abertos no Google Colab**

Bruno Gimenes Delphim

[*'Discente do curso de Biblioteconomia e Ciência da Informação - Campus USP-RP*](https://github.com/bgd94)

Neste notebook do Google Colab, eu vou mostrar como utilizar Python para acessar dados abertos do governo.

---

## Python

## catalogos-dados-brasil (**tableschema & datapackage**)
Conforme https://github.com/dadosgovbr/catalogos-dados-brasil/blob/master/scripts/uso/como-usar-com-o-pandas.ipynb

### Utilizando o terminal/prompt de comando para instalar pacotes Python externos

In [ ]:
pip install datapackage tableschema-pandas plotly plotly_express

#### Imprimindo versões instaladas

In [ ]:
pip freeze | grep -e datapackage -e tableschema -e plotly

### Lendo o pacote de dados (**plotly**)

In [ ]:
import pandas as pd

# Para trabalhar com Frictionless Data – frictionlessdata.io
from tableschema import Storage
from datapackage import Package

# Para visualização
import plotly_express as px
import plotly as py, plotly.graph_objects as go


# Tentar gravar no Panda e imprimir "true" se for bem-sucedido
output = "true"
try:
    url = 'https://github.com/dadosgovbr/catalogos-dados-brasil/raw/master/datapackage.json'

    # Conectar ao armazenamento Pandas
    storage = Storage.connect('pandas')

    # Carregar o pacote de dados da URL e salvar no armazenamento
    package = Package(url)
    package.save(storage=storage)

    # Se não houver exceções, imprimir "true"


except Exception as e:
    # Se ocorrer uma exceção, imprimir "false" e a mensagem de erro
    output = "false"
    print(e)

output

storage.buckets

type(storage['catalogos'])

storage['solucao']

storage['catalogos'].head()

tipo_solucao = storage['catalogos'].groupby('Solução').count()['URL'].rename('quantidade')
tipo_solucao

px.bar(
    pd.DataFrame(tipo_solucao).reset_index(),
    x = 'Solução',
    y = 'quantidade',
    color = 'Solução',
    color_discrete_sequence = py.colors.qualitative.Set2
)

poder = storage['catalogos'].groupby('Poder').count()['URL'].rename('quantidade')
poder

go.Figure(
    data=go.Pie(
        labels=poder.index,
        values=poder.values,
        hole=.4
    )
).show()

esfera = storage['catalogos'].groupby('Esfera').count()['URL'].rename('quantidade')
esfera

go.Figure(
    data=go.Pie(
        labels=esfera.index,
        values=esfera.values,
        hole=.4
    )
).show()

uf = storage['catalogos'].groupby('UF').count()['URL'].rename('quantidade')
uf

px.bar(
    pd.DataFrame(uf).reset_index(),
    x = 'UF',
    y = 'quantidade',
    color = 'UF',
    color_discrete_sequence = py.colors.qualitative.Set3
)

# Filtrar os registros com campo "UF" igual a "SP"
filtered_data = storage['catalogos'][storage['catalogos']['UF'] == 'SP']

# Exibir os registros filtrados
filtered_data

## excercicio-dados-sp (**frictionless**)


### 1. Instala pacotes Python externos

In [ ]:
pip install frictionless plotly plotly_express db-sqlite3

#### Imprime versões instaladas

In [ ]:
pip freeze | grep -e frictionless -e plotly -e db-sqlite3

### 2. Converte arquivo XLSX em CSV

In [3]:
import pandas as pd

In [4]:
def convert_excel_to_csv(path, file, extension):
  content = pd.read_excel(f"{path}/{file}.{extension}")

  content.to_csv(f"{file}.csv", mode = "a", index = None, header = True)

  return f"{file}.csv"

In [ ]:
csv_file_path = convert_excel_to_csv("/content", "IDESP_ESCOLA_2022", "xlsx")
csv_file_path

### 3. Valida arquivo CSV

In [6]:
from frictionless import validate

In [ ]:
# Descrevendo os dados

#description = describe(csv_file_path)
#pprint(description)

# Ajustar metadados caso necessário (ex.: field_missing_values, resource.schema.foreign_keys -> .yaml)

# Extraindo dados

#rows = extract(csv_file_path)
#pprint(rows)

#first_or_default = list(rows.values())[0]
#print(first_or_default)

In [ ]:
# Validando dados

report = validate(csv_file_path)
report

### 4. Cria banco de dados SQL local

In [8]:
# Carrega extensao SQL no ambiente

%load_ext sql

In [9]:
# Cria arquivo de banco de dados SQLite local -> PostgreSQL seria mais adequado
# para aplicativos de grande escala e necessidades de segurança avançadas.
%%sql
sqlite:///project.db

### 5. Carrega arquivo CSV no banco de dados SQL

In [10]:
from frictionless import formats, Resource

In [ ]:
# Cria recurso no formato 'Data Resource' a partir do arquivo .csv validado

resource = Resource(csv_file_path)
resource

In [ ]:
# Indexa/carrega dados do recurso 'Data Resource' na tabela 'idesp' do banco de dados

resource.index('sqlite:///project.db', name='idesp')
Resource('sqlite:///project.db', control=formats.sql.SqlControl(table='idesp')).extract()

### 6. Conecta no banco de dados SQL e seleciona registros

In [ ]:
# Seleciona primeiros 3 registros da tabela 'idesp' no banco de dados

#%%sql
#select * from idesp limit 3;

In [13]:
import sqlite3
from datetime import date

In [14]:
db = sqlite3.connect("project.db")

In [15]:
db = sqlite3.connect("project.db")
query = """
  select * from idesp
"""

### 7. Cria data-frame e gráficos a patir dos registros selecionados

In [ ]:
df = pd.read_sql_query(query,db)
df.head(5)

In [ ]:
import numpy as np
import plotly.express as px

# Plot 1: Pairplot using Plotly
fig = px.scatter_matrix(df)
fig.update_layout(title=f'Gráfico de Pares do Conjunto de Dados Desconhecido "{csv_file_path}"')
fig.show()

# Plot 2: Histogram of a numerical column
numerical_column = df.select_dtypes(include=[np.number]).columns[0]  # Select the first numerical column
fig = px.histogram(df, x=numerical_column, nbins=10, title=f'Histograma de {numerical_column}')
fig.show()

## Referências
 - SANTAREM SEGUNDO, Jose Eduardo. *Data Science para Humanas, Introdução a análise de dados, modelos e algoritmos de Machine Learning*. 26 de março de 2024. Faculdade de Filosofia, Ciências e Letras de Ribeirão Preto, Ribeirão Preto, SP
 - https://www.youtube.com/dataprofessor (acesso em 03/04/2024)
 - https://github.com/dadosgovbr/catalogos-dados-brasil/blob/master/scripts/uso/como-usar-com-o-pandas.ipynb (acesso em 03/03/2024)
 - ChatGPT-4 (diversos prompts diferentes)
 - https://framework.frictionlessdata.io/ (acesso em 04/04/2024)
 - https://specs.frictionlessdata.io/data-resource/#language (acesso em 05/04/2024)
 - https://thivyapriyaa.medium.com/google-colab-series-setting-up-sqlite-de2aaaa02ec5 (acesso em 05/04/2024)
 - https://medium.com/@raftaarrashedin100/sqlite-connection-using-python-in-google-colab-6c8130fd672d (acesso em 05/04/2024)